# data processing

## json create

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
# package import

import pandas as pd
from data_pack import DataSetDefinition
import utils_cpu
import utils_gpu

from typing import List, Dict
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset_def = DataSetDefinition(
    processed_dir_suffix="4x"
)

In [5]:
dataset_def.processed_dir_suffix

'4x'

In [6]:
step2_split = dataset_def.step2_dict_generator(with_radiance_hint=True)

In [7]:
train_split = step2_split["train"]; test_split = step2_split["eval"]

In [8]:
train_split_df = pd.DataFrame(train_split); test_split_df = pd.DataFrame(test_split)

In [9]:
train_split_df

,image_id,viewpoint_id,lighting_condition_id,image_path,mask_path,output_dir,fov,mask_threshold,env_map,pls,use_gpu_for_rendering,resolution,hint_diffuse_path,hint_ggx0.05_path,hint_ggx0.13_path,hint_ggx0.34_path
0,obj_01_car,NE7,011,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/NE7...,./data/processed/4x/train/hints/obj_01_car/NE7...,./data/processed/4x/train/hints/obj_01_car/NE7...,./data/processed/4x/train/hints/obj_01_car/NE7...
1,obj_01_car,NA3,009,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/009.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/NA3...,./data/processed/4x/train/hints/obj_01_car/NA3...,./data/processed/4x/train/hints/obj_01_car/NA3...,./data/processed/4x/train/hints/obj_01_car/NA3...
2,obj_01_car,NE7,006,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/006.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/NE7...,./data/processed/4x/train/hints/obj_01_car/NE7...,./data/processed/4x/train/hints/obj_01_car/NE7...,./data/processed/4x/train/hints/obj_01_car/NE7...
3,obj_01_car,NA7,003,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/003.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/NA7...,./data/processed/4x/train/hints/obj_01_car/NA7...,./data/processed/4x/train/hints/obj_01_car/NA7...,./data/processed/4x/train/hints/obj_01_car/NA7...
4,obj_01_car,CB5,009,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/009.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/CB5...,./data/processed/4x/train/hints/obj_01_car/CB5...,./data/processed/4x/train/hints/obj_01_car/CB5...,./data/processed/4x/train/hints/obj_01_car/CB5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,obj_64_greenhead,NE7,013,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/013.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...
8188,obj_64_greenhead,NC3,009,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/009.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...
8189,obj_64_greenhead,CA2,012,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/012.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...
8190,obj_64_greenhead,NC3,008,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/008.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_g

In [9]:
# 같은 object, 같은 view_point에서 자기자신만 뺀 나머지의 light condition을 ref로서 한다.
# Step 1: Create the 'ref' column
def create_ref_column(df):
    # Group by 'image_id' and 'viewpoint_id'
    groups = df.groupby(['image_id', 'viewpoint_id'])
    refs = {}

    # Iterate over each group
    for (image_id, viewpoint_id), group in groups:
        # Create a list of image_paths and lighting_condition_ids for the group
        image_paths = group['image_path'].tolist()
        lighting_ids = group['lighting_condition_id'].tolist()

        # Iterate over each row in the group
        for idx, row in group.iterrows():
            # Exclude the current row's lighting_condition_id
            ref_paths = [
                p for lc_id, p in zip(lighting_ids, image_paths)
                if lc_id != row['lighting_condition_id']
            ]
            refs[idx] = ref_paths

    # Map the refs to the DataFrame
    df['ref'] = df.index.map(refs)
    return df

In [11]:
train_split_df[(train_split_df["viewpoint_id"] == "CF8")]

,image_id,viewpoint_id,lighting_condition_id,image_path,mask_path,output_dir,fov,mask_threshold,env_map,pls,use_gpu_for_rendering,resolution,hint_diffuse_path,hint_ggx0.05_path,hint_ggx0.13_path,hint_ggx0.34_path
11,obj_01_car,CF8,002,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/002.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...
23,obj_01_car,CF8,001,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/001.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...
24,obj_01_car,CF8,007,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/007.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...
27,obj_01_car,CF8,004,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/004.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...
30,obj_01_car,CF8,006,./data/processed/4x/train/images/obj_01_car/ob...,./data/processed/4x/train/masks/obj_01_car/obj...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/006.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...,./data/processed/4x/train/hints/obj_01_car/CF8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8136,obj_64_greenhead,CF8,008,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/008.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...
8159,obj_64_greenhead,CF8,003,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/003.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...
8162,obj_64_greenhead,CF8,012,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/012.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...,./data/processed/4x/train/hints/obj_64_greenhe...
8172,obj_64_greenhead,CF8,002,./data/processed/4x/train/images/obj_64_greenh...,./data/processed/4x/train/masks/obj_64_greenhe...,./data/processed/4x/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/002.hdr,"[[0, 0, 0]]",True,128,./data/processed/4x/train/hints/obj

In [13]:
train_split_df_filtered = train_split_df[(train_split_df["viewpoint_id"] == "CF8") & (train_split_df["lighting_condition_id"] =="011")]
train_split_df_filtered

,image_id,viewpoint_id,lighting_condition_id,image_path,mask_path,output_dir,fov,mask_threshold,env_map,pls,use_gpu_for_rendering,resolution,hint_diffuse_path,hint_ggx0.05_path,hint_ggx0.13_path,hint_ggx0.34_path
124,obj_01_car,CF8,011,./data/processed/128_v3/train/images/obj_01_ca...,./data/processed/128_v3/train/masks/obj_01_car...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_01_car...,./data/processed/128_v3/train/hints/obj_01_car...,./data/processed/128_v3/train/hints/obj_01_car...,./data/processed/128_v3/train/hints/obj_01_car...
201,obj_02_egg,CF8,011,./data/processed/128_v3/train/images/obj_02_eg...,./data/processed/128_v3/train/masks/obj_02_egg...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_02_egg...,./data/processed/128_v3/train/hints/obj_02_egg...,./data/processed/128_v3/train/hints/obj_02_egg...,./data/processed/128_v3/train/hints/obj_02_egg...
276,obj_03_chicken,CF8,011,./data/processed/128_v3/train/images/obj_03_ch...,./data/processed/128_v3/train/masks/obj_03_chi...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_03_chi...,./data/processed/128_v3/train/hints/obj_03_chi...,./data/processed/128_v3/train/hints/obj_03_chi...,./data/processed/128_v3/train/hints/obj_03_chi...
456,obj_04_stone,CF8,011,./data/processed/128_v3/train/images/obj_04_st...,./data/processed/128_v3/train/masks/obj_04_sto...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_04_sto...,./data/processed/128_v3/train/hints/obj_04_sto...,./data/processed/128_v3/train/hints/obj_04_sto...,./data/processed/128_v3/train/hints/obj_04_sto...
596,obj_05_bird,CF8,011,./data/processed/128_v3/train/images/obj_05_bi...,./data/processed/128_v3/train/masks/obj_05_bir...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_05_bir...,./data/processed/128_v3/train/hints/obj_05_bir...,./data/processed/128_v3/train/hints/obj_05_bir...,./data/processed/128_v3/train/hints/obj_05_bir...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7568,obj_60-fabric-green-hedgehog,CF8,011,./data/processed/128_v3/train/images/obj_60-fa...,./data/processed/128_v3/train/masks/obj_60-fab...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_60-fab...,./data/processed/128_v3/train/hints/obj_60-fab...,./data/processed/128_v3/train/hints/obj_60-fab...,./data/processed/128_v3/train/hints/obj_60-fab...
7781,obj_61-fabric-dolphin,CF8,011,./data/processed/128_v3/train/images/obj_61-fa...,./data/processed/128_v3/train/masks/obj_61-fab...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_61-fab...,./data/processed/128_v3/train/hints/obj_61-fab...,./data/processed/128_v3/train/hints/obj_61-fab...,./data/processed/128_v3/train/hints/obj_61-fab...
7809,obj_62-fabric-birthday-cake,CF8,011,./data/processed/128_v3/train/images/obj_62-fa...,./data/processed/128_v3/train/masks/obj_62-fab...,./data/processed/128_v3/train/hints,None,0.25,./data/generate_light_gt_sg/hdrs/011.hdr,"[[0, 0, 0]]",True,128,./data/processed/128_v3/train/hints/obj_62-fab...,./data/processed/128_v3/train/hints/obj_62-fab...,./data/processed/128_v3/train/hints/obj_62-fab...,./data/processed/128_v3/train/hints/obj_62-fab...
8003,obj_63-fabric-friends-cup,CF8,011,./data/processed/128_v3/train/images/obj_63-fa...,./data/processed/128_v3/train/masks/obj_63-fab...,./data/processed/128_v3/train/hints,None,0.25,./data/generat

In [10]:
# directory structure -> list of data

step1_datadict_train_list: List[Dict[str, str]] = []
step1_datadict_eval_list: List[Dict[str, str]] = []

for each_class_name in dataset_def.class_name_list:
    for each_view_point in dataset_def.view_point_list:
        for each_light_condition in dataset_def.light_condition_list:

            image_path = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
            )
            mask_path = DataSetDefinition.get_mask_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                view_point=each_view_point,
            )

            parsed_name = DataSetDefinition.get_image_path(
                data_root_dir=dataset_def.raw_data_root_dir,
                class_name=each_class_name,
                light_condition=each_light_condition,
                view_point=each_view_point,
                parsed=True,
            )

            if parsed_name in dataset_def.eval_split:
                step1_datadict_eval_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_{each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )
            else:
                step1_datadict_train_list.append(
                    {
                        "key": f"{each_class_name}_{each_light_condition}_{each_view_point}",
                        "image_path": image_path,
                        "mask_path": mask_path,
                    }
                )

# caption generation

In [11]:
# 여기서 dict로 for문 돌려버리자.
# 64개의 object에 대해서 가장 광량이 높은 조건으로 가져오도록 한다.
# obj_01_car_013_CF8.png
captioner = utils_gpu.BLIPI2TLarge()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.31s/it]


In [12]:
priority_mapping = {'013': 1, '012': 2}
train_split_df['priority'] = train_split_df['lighting_condition_id'].map(priority_mapping).fillna(3)
train_split_df_sorted = train_split_df.sort_values(by=['image_id', 'viewpoint_id', 'priority'])
df_selected = train_split_df_sorted.drop_duplicates(subset=['image_id', 'viewpoint_id'], keep='first')
for index, row in tqdm(df_selected.iterrows(), total=len(df_selected)):
    caption = captioner(row['image_path'])
    df_selected.at[index, 'caption'] = caption

  0%|          | 0/640 [00:00<?, ?it/s]/tmp/ipykernel_249726/4280418720.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected.at[index, 'caption'] = caption
  4%|▍         | 24/640 [00:46<19:36,  1.91s/it]

100%|██████████| 640/640 [19:39<00:00,  1.84s/it]


In [13]:
caption_path = os.path.join("/data2/code/diffusion-project/data", "captions_raw_4x_large_v2.csv")
df_selected.to_csv(caption_path, index=False)

In [26]:
df_selected.iloc[0]

image_id                                                        obj_01_car
viewpoint_id                                                           CA2
lighting_condition_id                                                  013
image_path               ./data/processed/128_v2/train/images/obj_01_ca...
mask_path                ./data/processed/128_v2/train/masks/obj_01_car...
output_dir                             ./data/processed/128_v2/train/hints
fov                                                                   None
mask_threshold                                                        0.25
env_map                           ./data/generate_light_gt_sg/hdrs/013.hdr
pls                                                            [[0, 0, 0]]
use_gpu_for_rendering                                                 True
resolution                                                             128
hint_diffuse_path        ./data/processed/128_v2/train/hints/obj_01_car...
hint_ggx0.05_path        

In [14]:
trimmed_df = df_selected.drop(columns=["image_path", "mask_path", "output_dir",
                          "fov", "mask_threshold", "env_map", "pls",
                          "use_gpu_for_rendering", "priority",
                          "hint_diffuse_path", "hint_ggx0.05_path", "hint_ggx0.13_path", "hint_ggx0.34_path"])

In [15]:
trimmed_df["caption"] = trimmed_df['caption'].str.replace('\n', '')

In [16]:
caption_path = os.path.join("/data2/code/diffusion-project/data", "captions_4x_large_v1.csv")
trimmed_df.to_csv(caption_path, index=False)

In [ ]:
"""
dataset composer

/data1/common_datasets/openillumination/processed/128_v3/train/images/obj_01_car/obj_01_car_011_NE7.png

/data1/common_datasets/openillumination/processed/128_v3/train/masks/obj_01_car/obj_01_car_CA2.png
-> /data1/common_datasets/openillumination/processed/128_v3/train/masks/obj_01_car/{view_point}/mask.png


/data1/common_datasets/openillumination/processed/128_v3/train/hints/obj_01_car/CA2/001/hint00_diffuse.png
/data1/common_datasets/openillumination/processed/128_v3/train/hints/obj_01_car/CA2/001/hint00_diffuse.png, hint00_ggx0.05.png, hint00_ggx0.13.png, hint00_ggx0.34.png
"""

In [17]:
"""

/data1/common_datasets/openillumination/processed/128_v3/train/hints/

하위경로를 recursive하게 볼 수 있어야 함.
view_point_light condition으로 새롭게 folder 구조 만들 필요가 있음

hint00_diffuse.png, hint00_ggx0.05.png, hint00_ggx0.13.png, hint00_ggx0.34.png을 gt_{}.png로 변환하고 processed_data_root/{class_name}/{view_point}/{light_condition}/gt_*.png로 복사하고 싶다. 또한 {data_root}/{class_name}/{class_name}{light_condition}{view_point}.png의 이미지를 processed_data_root/{class_name}/{view_point}/{light_condition}/gt.png로 복사하고 싶다.



"""
import os
import shutil
from tqdm import tqdm

def list_files_recursive(directory, filter_keyword=None):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for name in files:
            if filter_keyword is None:
                file_list.append(os.path.join(root, name))
            else:
                if filter_keyword in name:
                    file_list.append(os.path.join(root, name))
                else:
                    pass
        # for name in dirs:
        #     file_list.append(os.path.join(root, name))
    return file_list


def copy_and_rename_files(data_root, processed_data_root, class_name, view_point, light_condition, each_hint_base_name, move_mask):
    # 원본 파일 경로
    
    # 대상 디렉토리 생성
    target_dir = os.path.join(processed_data_root, class_name, view_point, light_condition)
    os.makedirs(target_dir, exist_ok=True)
    
    # hint 파일 복사 및 이름 변경
    src = os.path.join(data_root, "hints", class_name, view_point, light_condition, each_hint_base_name)
    
    diffuse_split = os.path.splitext(each_hint_base_name)[0].split("_")[1]
    dst = os.path.join(target_dir, f"gt_{diffuse_split}.png")
    
    # import ipdb; ipdb.set_trace()
    
    shutil.copy(src, dst)
    
    # 추가 파일 복사 및 이름 변경
    additional_file = f"{class_name}_{light_condition}_{view_point}.png"
    src = os.path.join(data_root, "images", class_name, additional_file)
    dst = os.path.join(target_dir, "gt.png")
    shutil.copy(src, dst)
    
    if move_mask:
        mask_sorce_dir = os.path.join(data_root, "masks", class_name)
        mask_target_dir = os.path.join(processed_data_root, "masks", class_name, view_point)
        os.makedirs(mask_target_dir, exist_ok=True)
        file_name = f"{class_name}_{view_point}.png"
        src = os.path.join(mask_sorce_dir, file_name)
        dst = os.path.join(mask_target_dir, f"mask.png")
        shutil.copy(src, dst)

In [18]:
train_dir_str = "/data2/code/diffusion-project/data/processed/4x/train/hints"
test_dir_str = "/data2/code/diffusion-project/data/processed/4x/eval/hints"

test_list = list_files_recursive(test_dir_str)
train_list = list_files_recursive(train_dir_str)

split_dict = {
    "train": train_list,
    "eval": test_list,
}

In [19]:
for each_split, split_list in split_dict.items():
    for each_hint in tqdm(split_list):
        split_name = each_hint.split("/")
        class_name = split_name[-4]
        view_point = split_name[-3]
        light_condition = split_name[-2]
        file_name = split_name[-1]
        move_mask = False
        if file_name == "hint00_diffuse.png":
            move_mask = True
        copy_and_rename_files(f"/data2/code/diffusion-project/data/processed/4x/{each_split}", f"/data2/code/diffusion-project/data/processed/512_v1_directory_processed/{each_split}", class_name, view_point, light_condition, file_name, move_mask)
        

 19%|█▊        | 6081/32768 [00:06<00:28, 941.14it/s] 

100%|██████████| 512/512 [00:02<00:00, 196.07it/s]


In [39]:
trimmed_df.columns

Index(['image_id', 'viewpoint_id', 'lighting_condition_id', 'resolution',
       'caption'],
      dtype='object')

In [ ]:
"""
json을 만들어야 한다.
{
  "image": "{root_dir}/{each_split}/{class_name}/{view_point}/{light_condition}/gt.png",
  "hint": [
    "{root_dir}/{each_split}/{class_name}/{view_point}/{light_condition}/gt_diffuse.png",
    "{root_dir}/{each_split}/{class_name}/{view_point}/{light_condition}/gt_ggx0.05.png"
    "{root_dir}/{each_split}/{class_name}/{view_point}/{light_condition}/gt_ggx0.13.png"
    "{root_dir}/{each_split}/{class_name}/{view_point}/{light_condition}/gt_ggx0.34.png"
  ],
  "ref": [
    "{root_dir}/{each_split}/{class_name}/{view_point}/{light_condition}/gt.png",
    "/absolute/path/to/your/file/view_0/env_0/gt.png",
    "/absolute/path/to/your/file/view_0/env_1/gt.png",
    "..."
  ],
  "text": "some text description generated by BLIP2"
  "mask": "{root_dir}/{each_split}/masks/{class_name}/{view_point}/mask.png"
  
}
/data1/common_datasets/openillumination/processed/128_v3_directory_processed/train/obj_01_car/CA2/001/

이때 ref의 경우에는 같은 view에서 다른 light condition을 list에서 filter를 하면 될 것이다.
"""





In [20]:
train_dir_str = "/data1/common_datasets/openillumination/processed/512_v1_directory_processed/train/"
test_dir_str = "/data1/common_datasets/openillumination/processed/512_v1_directory_processed/eval/"


test_list = list_files_recursive(test_dir_str, filter_keyword="gt.png")
train_list = list_files_recursive(train_dir_str, filter_keyword="gt.png")

split_dict = {
    "train": train_list,
    "eval": test_list,
}

In [24]:

def replace_filename(full_path, new_filenames):
    # 디렉토리 경로와 파일 이름 분리
    dir_path, _ = os.path.split(full_path)
    
    # 새로운 파일 경로 생성
    new_paths = [os.path.join(dir_path, new_filename) for new_filename in new_filenames]
    return new_paths

def get_available_light_conditions(root_dir, split_name, class_name, view_point, exclude_light_condition=None):
    base_path = os.path.join(root_dir, split_name, class_name, view_point)
    light_conditions = []
    
    if os.path.exists(base_path):
        for light_condition in os.listdir(base_path):
            if light_condition == exclude_light_condition:
                continue  # 자기 자신은 제외
            light_condition_path = os.path.join(base_path, light_condition)
            image_path = os.path.join(light_condition_path, 'gt.png')
            if os.path.exists(image_path):
                light_conditions.append(light_condition)
    return light_conditions

def get_images_for_all_light_conditions(root_dir, split_name, class_name, view_point, exclude_light_condition=None):
    light_conditions = get_available_light_conditions(root_dir, split_name, class_name, view_point, exclude_light_condition)
    image_paths = []
    for light_condition in light_conditions:
        image_path = os.path.join(split_name, class_name, view_point, light_condition, 'gt.png')
        image_paths.append(image_path)
    return image_paths

def each_elem_generator(file_path, root_dir, trimmed_df, upper_dir=None):
    
    hints = [
    "gt_diffuse.png",
    "gt_ggx0.05.png",
    "gt_ggx0.13.png",
    "gt_ggx0.34.png",
]
    
    if 'train' in file_path:
        split_name = "train"
        rel_path = file_path.split('train', 1)[-1].lstrip('/')
    elif 'eval' in file_path:
        split_name = "eval"
        rel_path = file_path.split('eval', 1)[-1].lstrip('/')
    else:
        raise ValueError(f"not valid split")
    
    # image_name
    replaced_name = os.path.join(split_name, rel_path)
    # train/obj_60-fabric-green-hedgehog/NA7/012/gt.png
    
    light_condition = rel_path.split("/")[-2]
    view_point = rel_path.split("/")[-3]
    class_name = rel_path.split("/")[-4]
    
    # hint
    hint_paths = replace_filename(replaced_name, hints)
    # masks
    mask_path = os.path.join(split_name, "masks", class_name, view_point, "mask.png")
    # caption
    caption = trimmed_df[(trimmed_df["image_id"] == class_name) & (trimmed_df["viewpoint_id"] == view_point)]["caption"].values[0]
    
    ref_list = get_images_for_all_light_conditions(root_dir, split_name, class_name, view_point, exclude_light_condition=light_condition)
    
    if upper_dir is not None:
        
        hint_paths = [os.path.join(upper_dir, each_hint) for each_hint in hint_paths]
        ref_list = [os.path.join(upper_dir, each_ref) for each_ref in ref_list]
        
        dict_elem = {
            "image": os.path.join(upper_dir, replaced_name),
            "hint": hint_paths,
            "mask": os.path.join(upper_dir, mask_path),
            "ref": ref_list,
            "text": caption,
        }
    else:
        dict_elem = {
            "image": replaced_name,
            "hint": hint_paths,
            "mask": mask_path,
            "ref": ref_list,
            "text": caption,
        }
    return dict_elem
    

In [45]:
trimmed_df

,image_id,viewpoint_id,lighting_condition_id,resolution,caption
104,obj_01_car,CA2,013,128,"a toy dump truck with a red, blue and yellow body"
33,obj_01_car,CB5,013,128,a toy dump truck with a red and blue body and ...
26,obj_01_car,CC7,013,128,a dump truck with a red and blue bucket
29,obj_01_car,CE2,013,128,a blue and green dump truck
120,obj_01_car,CF8,013,128,a toy dump truck with a red and blue bucket
...,...,...,...,...,...
8115,obj_64_greenhead,NA3,013,128,green mannequin head mask
8081,obj_64_greenhead,NA7,013,128,green mannequin head with dots
8149,obj_64_greenhead,NC3,013,128,a green mannequin head with dots on it
8186,obj_64_greenhead,NE1,013,128,3D printed head for a 3D printed body


In [25]:
data_dict = {
    "train": [],
    "eval": [],
}

root_dir = "/data1/common_datasets/openillumination/processed/512_v1_directory_processed/"

for each_split, split_list in split_dict.items():
    for each_file in tqdm(split_list):
        data_dict[each_split].append(each_elem_generator(each_file, root_dir, trimmed_df, upper_dir="dataset_512_v1"))

100%|██████████| 128/128 [00:00<00:00, 1814.51it/s]


In [86]:
data_dict["eval"]

[{'image': 'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/010/gt.png',
  'hint': ['dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/010/gt_diffuse.png',
   'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/010/gt_ggx0.05.png',
   'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/010/gt_ggx0.13.png',
   'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/010/gt_ggx0.34.png'],
  'mask': 'dataset_v2/eval/masks/obj_60-fabric-green-hedgehog/CF8/mask.png',
  'ref': ['dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/013/gt.png'],
  'text': 'dataset_v2/a green stuffed animal with a green hat and a green tail'},
 {'image': 'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/013/gt.png',
  'hint': ['dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/013/gt_diffuse.png',
   'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/013/gt_ggx0.05.png',
   'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/013/gt_ggx0.13.png',
   'dataset_v2/eval/obj_60-fabric-green-hedgehog/CF8/013/gt_ggx0.34.png'],
  'mask':

In [27]:
import json
for each_split, each_data in data_dict.items():
    with open(f"/data2/code/diffusion-project/data/processed/512_v1_directory_processed/{each_split}_v2.json", "w") as f:
        json.dump(each_data, f)

In [28]:
import json

for each_split, each_data in data_dict.items():
    # JSONL 파일로 저장
    with open(f"/data2/code/diffusion-project/data/processed/512_v1_directory_processed/{each_split}_v2.jsonl", "w") as f:
        for record in each_data:
            # 각 dict를 JSON으로 변환한 뒤 한 줄씩 저장
            f.write(json.dumps(record) + "\n")
